In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from huggingface_hub import login

**Hugging Face token login methods (Fine-Grained)**

In [ ]:
import os

#Edwin's Token
# os.environ['HUGGINGFACE_TOKEN'] = [token]
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = [token]

In [ ]:
# Load the PDF document

from langchain.document_loaders import PyPDFLoader

pdfloader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/RAG/RAG-data/Fostering_a_Positive_Workplace_Culture_Impacts_on_.pdf")
documents = pdfloader.load()

In [ ]:
len(documents)

18

In [ ]:
# Split the PDF into manageable chunks

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [ ]:
# Load the embedding model

from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})

<ipython-input-6-b75bea39d7c8>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still o

In [ ]:
# Create a FAISS vector store for efficient retrieval & a retriver

from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embedding_model)

retriever = db.as_retriever()

In [ ]:
# Load the google/gemma-7b

from langchain.llms import HuggingFaceHub

repo_id = "google/gemma-7b"
llm = HuggingFaceHub(
    repo_id=repo_id,
    model_kwargs={"temperature": 0.7, "max_length": 128}
)

<ipython-input-8-16f1ba8782aa>:6: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


In [ ]:
import time

# Retry mechanism with generic exception handling
max_retries = 5
for attempt in range(max_retries):
    try:
        # Invoke the model (replace `qa_chain.invoke` with your actual method)
        response = llm.invoke("How does communication impact the workplace culture?")
        print(f"Response: {response}")
        break  # Exit loop if successful
    except Exception as e:  # Catch any error
        wait_time = 2 ** attempt  # Exponential backoff
        print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
        time.sleep(wait_time)
else:
    print("Failed after multiple retries.")

Error occurred: 500 Server Error: Internal Server Error for url: https://api-inference.huggingface.co/models/google/gemma-7b (Request ID: xg3d6l8TOjF3FiYRS7NwA)

Model too busy, unable to get response in less than 60 second(s). Retrying in 1 seconds...
Response: How does communication impact the workplace culture? The answer is simple: it impacts every aspect of your company. From the daily grind to the long-term vision, everything your organization does is impacted by how well you communicate with your people. 

But what happens when you don’t communicate well? You’d be surprised at how often people overlook the importance of effective communication in the workplace. But if you’re looking for ways to improve your workplace culture, it’s time to focus on communication. 

By investing in your communication channels, you can create a more positive work environment that will benefit everyone involved. Here are just a few ways your company can benefit from improving communication in the wo

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """Based on the provided context, respond to the question below while adhering to these guidelines:
1. If the answer is unclear or not found, do not speculate. Instead, state, "I do not know the answer"
2. If the answer is found, provide a clear and concise response in no more than ten sentences.


Question: {question}

Answer:

{context}
"""


PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
# Set up the retrieval-based QA chain

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    # return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [ ]:
import time

# Retry mechanism with generic exception handling
max_retries = 5
for attempt in range(max_retries):
    try:
        # Invoke the model (replace `qa_chain.invoke` with your actual method)
        response = qa_chain.invoke({"query": "What are the benefits of creating a positive workplace culture for an organization?"})
        print(f"Response: {response}")
        break  # Exit loop if successful
    except Exception as e:  # Catch any error
        wait_time = 2 ** attempt  # Exponential backoff
        print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
        time.sleep(wait_time)
else:
    print("Failed after multiple retries.")

Error occurred: 500 Server Error: Internal Server Error for url: https://api-inference.huggingface.co/models/google/gemma-7b (Request ID: SFw4Lhv9fNkWaALAVLDMX)

Model too busy, unable to get response in less than 60 second(s). Retrying in 1 seconds...
Error occurred: 500 Server Error: Internal Server Error for url: https://api-inference.huggingface.co/models/google/gemma-7b (Request ID: te6G8ow_EvOVQIA-6_0cI)

Model too busy, unable to get response in less than 60 second(s). Retrying in 2 seconds...
Response: {'query': 'What are the benefits of creating a positive workplace culture for an organization?', 'result': 'Based on the provided context, respond to the question below while adhering to these guidelines:\n1. If the answer is unclear or not found, do not speculate. Instead, state, "I do not know the answer"\n2. If the answer is found, provide a clear and concise response in no more than ten sentences.\n\n\nQuestion: What are the benefits of creating a positive workplace culture f

In [ ]:
print(f"Answer: {response['result']}")

Answer: Based on the provided context, respond to the question below while adhering to these guidelines:
1. If the answer is unclear or not found, do not speculate. Instead, state, "I do not know the answer"
2. If the answer is found, provide a clear and concise response in no more than ten sentences.


Question: What are the benefits of creating a positive workplace culture for an organization?

Answer:

Human Resource Management - An Update
2
development, and collaborative dynamics, organizations can unlock the latent 
capabilities within their workforce, resulting in elevated levels of performance and 
productivity.
2. Cultivating a positive workplace culture: the catalyst for success
2.1  Positive psychology and positive intelligence: foundations for a positive 
organizational culture
A positive workplace culture encompasses a range of components that foster a 
harmonious and productive working atmosphere. While it is not synonymous with 
positive psychology or positive intelligenc

In [ ]:
!pip install langchain
!pip install huggingface_hub
!pip install pypdf
!pip install faiss-cpu
!pip install sentence-transformers
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.9 MB/s eta 0:00:00
